# 2. Decoding

Decoding has the objective to 'decode' the neural stimulus from the neural activity. This requires that the activity of neurons varies during stimulus presentation and therefore this approach is routinely used by neuroscientists to see if a neural population could be involved in processing a particular stimulus (or other behavioral variable). The classical approach is to train a model, i.e. classifier, on the neural activity such that you can use the model to decode the stimulus identity from 'unseen' activity patterns.

The neural activity comes from an experiment where data was recorded from awake, head-fixed mouse using two neuropixel probes for 1.5 hours. See [link](http://data.cortexlab.net/dualPhase3/) for more details and the original dataset.

Here are the relevant details of the data for the decoding task. 

* Neural data is the spike count binned in 100ms of 76 V1 neurons
* During the recording, 17 different visual stimuli were shown to the mouse, each repeated 10 times in random order for 1.5 seconds (30 bins)
* The stimuli are numbered with the first 16 numbers correspond to orientations 0, 22.5, ..., 337.5. Stimulus number 17 is darkness
*  The data recording starting 0.5 seconds before stimulus onset until 0.5 seconds after stimulus offset.


In a first step (2.1) we select the neural activity in the presence of the stimulus.
Secondly (2.2) we provide example code to train a classifier.

Your objective (2.3) is to decode the same neural data as in the example but by using Support Vector Machine (SVM) as a classifier.



In [ ]:
# standard imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# loading the data
import pickle

## 2.1 Loading the data

The cell below loads an array X of size (trials, time points, neurons) containing the neural activity. The number of trials is 170 (10 repetitions per stimuli for 17 stimuli), the number of time points is 25 (0.5 seconds before,1.5 sec stimulus, 0.5 second after = 2.5 seconds with bins of 100 ms), and the number of neurons is 76. 

We further define an array Y of size (trials,) containing the stimulus identity of each trial. Since we have 170 trials we also have 170 stimulus identities. We will start by predicting the stimulus ($y$) from activity in the 1000-1100 ms bin after stimulus onset.

In [ ]:
with open('decoding_data.pickle', 'rb') as handle:
    data_dict = pickle.load(handle)

    
X = data_dict['activity']
y = data_dict['stimulus']
trials, time_bins, neurons = X.shape
n_stim = len(np.unique(y))

print("Activity X: %u trials, %u bins, %u neurons"%(trials, time_bins, neurons))

print("Labels y: %u trials,  %u stimuli"%(trials, n_stim))

## 2.2 Classifiy the neural data according to stimulus identity
### 2.2.1 Linear classifier as example
Linear regression predicts a stimulus $y_i$ by a $\hat{y}_i$, a weighted sum of neural activity:
$$ \hat{y_i} = \sum_{j=1}^n \beta_j x_{ij}.
$$
Here the sum is over the neurons $j=1,...,n$. The index $i$ denotes the sample. To fit a linear regression model means to find the $\beta_i$'s that minimize the difference between the true stimulus and the estimated stimulus $\hat{y}$:
$$
\sum_i (y_i - \hat{y}_i)^2  = \sum_{i} (y_i - \sum_j \beta_j x_{ij})^2
$$
It turns out that simply minimizing the error is not what we want. Why? Because our linear regression will use any correlation between activities $x$ and stimuli $y$. Maybe there are some noisy neurons that just happen to be correlated with the stimulus by chance. To make sure our decoder only uses the neurons it needs, we penalize it for the weights $\beta$ that are different from 0. Our objective now becomse
$\hat{y}$:
$$
\sum_i (y_i - \hat{y}_i)^2  + \alpha \sum_j \beta_j^2.
$$
This version of linear regression is called ridge regression. 
By minimizing these two terms, we force the model to minize the error by assigning weights $\beta$ to only those neurons it really needs. The 'tuning parameter' $\alpha$ determines how important small weights are compared to the error minimization.






### 2.2.2 Applying the ridge regression classifier (naively)

The following code fits ridge regression to the data using the the implementation from the scikit-learn library. 
See https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge for more information. 

In [ ]:
from sklearn.linear_model import Ridge
alpha = 10 # ad hoc choise of regularization strength
t = 15 # activity in the 1000-1100 ms bin after stimulus onset.
ridge = Ridge(alpha)
ridge.fit(X[:,t], y)
y_hat = ridge.predict(X[:,t])

Next we need a way of assessing how good our model does. We could use the squared error, but this isn't always
easy to interpret or compare between datasets. We will therefore use the coefficient of determination $R^2.$ This measures the fraction of the variability in the stimulus that is explained by our prediction. If our classifier does perfect, the $R^2$ will be 1. If the classifier does only as good (or worse) compared to predicting the average stimulus, the $R^2$ will be 0 (or lower). 

In [ ]:
def get_R2(y, y_hat):
    """ Fraction of variance explained
        y, y_hat : arrays of size n_trials, )
    """
    SS_tot = np.mean((y-y.mean())**2)
    SS_reg = np.mean((y-y_hat)**2)
    return 1-SS_reg/SS_tot
print("R2: %0.2f"%get_R2(y,y_hat))

We explained over half of the variability in the stimulus. Unfortunately there are two issues that need to be solved. The first issue is that we ad-hoc choose a value of or regularization strength  α
 . Maybe there are other values of  α

  that allow us to do better! The second issue is that we trained and tested our model on the same dataset. This likely leads to overfitting, meaning a model that uses the noise particular to the activity pattterns in this dataset to predict the stimulus. It will therefore not generalize if the noise changes. We will now adress both issues.

### 2.2.3 Rigorous application of the ridge regression classifier by using training, validation and test sets

**a. Split the data**

First we will split the data into training, validation and test sets.The training subset is for training, while the validation subset is for assessing the performance of a model with a particular alpha value. Of course the best alpha value could be depending on the choise of the validation set. So to test the performance of the 'best performing' model we will need a third data set (test set).

The following code splits the data into these three subsets. The first 102 trials (6 repeats of 17 stimulus presentations) comprise the training set, the next 34 trials (2 repeats of 17 stimulus presentations) comprise the validation set and the final 34 trials (2 repeats of 17 stimulus presentations) comprise the test set.

In [ ]:
n_train = 102
n_valid = 34
n_test = 34

# choose the trials
train_idx = np.arange(n_train)
valid_idx = np.arange(n_train, n_train+n_valid)
test_idx = np.arange(n_train+n_valid, trials)

# Split the data
X_train = X[train_idx]
y_train = y[train_idx]
X_valid = X[valid_idx]
y_valid = y[valid_idx]
X_test = X[test_idx]
y_test = y[test_idx]

**b. Fitting a linear classifier** 

In [ ]:
t = 15 # activity in the 1000-1100 ms bin after stimulus onset.
alphas = np.logspace(-1,3,100)
R2s = np.zeros((len(alphas), ))
for i, alpha in enumerate(alphas):
    # Your code for fitting ridge regression goes here
    # store validation shore in R2s[i]


**c. Choose the optimal alpha value** 

Plot R2 as a function of alpha and choose the alpha that gives the highest R2 on the validation set. Use this to 
compute the performance on the train, validation and test set. Which is highest? 

In [ ]:
# After you have fitted a ridge regression for each alpha,
# this code will show their performance
plt.semilogx(alphas, R2s)
plt.xlabel('alphas')
plt.ylabel('R2s')
alpha_opt = alphas[np.argmax(R2s)]
ridge = Ridge(alpha_opt)
ridge.fit(X_train[:,t], y_train)
y_train_hat = ridge.predict(X_train[:,t])
y_valid_hat = ridge.predict(X_valid[:,t])
y_test_hat = ridge.predict(X_test[:,t])
print('R2 training set',get_R2(y_train, y_train_hat))
print('R2 validation set',get_R2(y_valid, y_valid_hat))
print('R2 test set',get_R2(y_test, y_test_hat))

### 2.2.4 Rigorous application of the ridge regression classifier with k-fold cross-validation

In the previous setup, we selected one subset of the data to train and another one to validate. This is a bit 'wasteful', since we are not using the validation data set for training our model. So we will change strategies and instead of splitting the data into training, validation and test sets, we only split the data into a training and test set. Just as before, we will use the test set to evaluate the performance of the model with the optimal alpha value.

To make best use of the training data we will do k-fold cross-validation. If you are not familiar with cross-validation, then have a look at the [following](https://www.youtube.com/watch?v=TIgfjmp-4BA) video for a better intuition. Briefly we split the training set into $K$ different subsets or 'folds'. We cycle over the different folds, each time leaving one of them out as a validation set and training the model on the remaining folds. At the end, we have $K$ scores and take the average. We finally choose the $\alpha$ that achieves the best average score. This may sound somewhat complicated at first. Luckily, `sklearn.linear_model` contains a class called `RidgeCV` that implements K-fold cross validation for us. 

In [ ]:
# Now we have two split by merging the prevous train and validation set
n_train = 102+34
# choose the trials
train_idx = np.arange(n_train)
# Split the data
X_train = X[train_idx]
y_train = y[train_idx]

In [ ]:
# Let sklearn do the cross validation for us, for the same time point
from sklearn.linear_model import RidgeCV
t = 15
K = 10 #The number of 'folds'
ridge = RidgeCV(alphas, cv=K)
ridge.fit(X_train[:,t], y_train)
y_train_hat = ridge.predict(X_train[:,t])
y_test_hat = ridge.predict(X_test[:,t])
print('R2 training set',get_R2(y_train, y_train_hat))
print('R2 test set',get_R2(y_test, y_test_hat))

### 2.2.5 Decoding over time

So far we only used the neural activity of one particular point in time (the 1000-1100 ms bin after stimulus onset) to decode the identity of the stimulus. However there could be other time points that encode more information about the stimulus. Remember, the stimulus is present from 0 to 1500 ms. In the next section we will evaluate which time point are most informative about the stimulus.

In [ ]:
from sklearn.linear_model import RidgeCV

R2s = np.zeros((time_bins, ))
for t in range(time_bins):
    # Your code goes here:

In [ ]:
# Running this will show the performance over time
time=np.linspace(0,2,time_bins)
plt.plot(time, R2s)
plt.hlines(0, time[0], time[-1],linestyles=":")
plt.xlabel("Time (s)", size=15)
plt.ylabel(r"$R^2$", size=15)
plt.show()

## Exercise: use another decoder

In the above example, we used a variant of linear regression, the simplest decoder possible. This was enough to decode the stimulus identity during stimulus presentation, but it is possible that another decoder can work better. For example, the set of stimuli is discrete so we could use a classifier such as an SVM. 

Tip: Most models don't have an Cross Validation (CV) extension (such as RidgeCV). However, you can use
`from sklearn.model_selection.cross_val_score `. 

In [ ]:
from sklearn.svm import SVC
# there's no CV extension of SVC so u
from sklearn.model_selection import cross_val_score 
c_vals = np.logspace(-1,2)
c_opts = np.zeros((time_bins, ))
accuracies = np.zeros((time_bins, ))

for t in range(time_bins):
    c_accuracies = np.zeros((len(c_vals),))
    for i, c in enumerate(c_vals):
        # Fit an SVM for C=c, store the performance in 
        # c_accuracies
        
    # choose best c and get test acc

In [ ]:
# Show classification accuracy over time

plt.plot(time, accuracies, 'o-', label='accuracy')
n_classes = len(np.unique(y_test))
plt.hlines(1/n_classes, time[0], time[-1],linestyles=":", label='chance')
plt.legend()
plt.xlabel("Time (m)")

In [ ]:
from sklearn.metrics import confusion_matrix
plt.imshow(confusion_matrix(y_test, y_test_hat))

## Extra Exercise: decode from single neurons

Here we used the activity patterns of the whole population to decode the stimulus. You may be wondering how well a classifier would do when it only had access to a single neuron. Try this out by training a decoder to predict 
the stimulus `y` from the activity single neurons stored in the last dimension of `X`.